In [4]:
from __future__ import division
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import quad
from scipy import interpolate
from astropy.io import fits
import matplotlib.pylab as plt
import os
import seaborn as sns
from scipy.interpolate import interp1d
sns.axes_style("darkgrid")
from MeasureFlux import FindCentroidX,gaussian, ShiftData, ShiftDataX
from scipy.ndimage.filters import gaussian_filter
from shutil import copyfile
%matplotlib inline

# Coadding Example: Flexure correction (wavelength shifting), Spatial shifting, then run through DCR cosmic ray removal, finally coadd

Note that this is for LowRedux reduced LRIS data

## First, shift the data to correct for the flexure and spatial dithering during the observations

In [2]:
centroids2 = np.array([2488,2488,2488,2486,2484,2483,2483,2481,2480])
centroid_shifts2 = centroids2-centroids2[4]
print centroid_shifts2

[ 4  4  4  2  0 -1 -1 -3 -4]


In [3]:
flexshift_1d = np.array([-12.16,-11.98,-11.66,-11.36,-11.70,-12.20,-12.46,-12.86,-13.10])
print np.round(flexshift_1d )
angperpix_1d = 0.256418793232
angperpix_2d = 0.258
flexshift_2d = -1*np.round((flexshift_1d*angperpix_1d)/angperpix_2d)
#the -1 is because of the definition of the shift value --> tested the other way around, was wrong, so need the -1
print flexshift_2d

[-12. -12. -12. -11. -12. -12. -12. -13. -13.]
[ 12.  12.  12.  11.  12.  12.  12.  13.  13.]


In [ ]:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])
refname = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science','sci-b150909_1092.fits')
hdulistref = fits.open(refname)
for fn in xrange(len(filenames)):
    
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug',filenames[fn])
    
    # copy the original file to shifted directory
    copyname = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',filenames[fn])
    copyfile(filename, copyname)
    
    
    hdulist = fits.open(copyname, mode='update')
    
    for ext in xrange(4):
        scidata = hdulist[ext].data
        sciref = hdulistref[ext].data
        scidatashiftedX = ShiftDataX(shift=centroid_shifts2[fn],dataref = sciref,datashift=scidata)
        scidatashiftedXY = ShiftData(shift=flexshift_2d[fn],dataref=sciref,datashift=scidatashiftedX)
        scidata[:,:] = scidatashiftedXY
    
    hdulist.flush() 
                      


    hdulist.close()
hdulistref.close()

## Now that the files have been shifted correctly, background subtract

In [ ]:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tempfile = os.path.splitext(tempfile)[0]
    tempfilen = tempfile + '_shifted_bgsub.fits'
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',filenames[fn])
    hdulist = fits.open(filename)
    scidata = hdulist[0].data
    header = hdulist[0].header
    skymodel = hdulist[2].data
    scinosky = scidata-skymodel
    hdulist.close()
    
    hdu = fits.PrimaryHDU()
    hdu.data = scinosky
    hdu.header = header
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tempfilen))
    hdulist.close()

### And save the standard deviation as a separate fits file

In [ ]:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tempfile = os.path.splitext(tempfile)[0]
    tempfilen = tempfile + '_std_shifted.fits'
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',filenames[fn])
    hdulist = fits.open(filename)
    header = hdulist[0].header
    invvar = hdulist[1].data
    std = np.sqrt(1/invvar)
    hdulist.close()
    
    hdu = fits.PrimaryHDU()
    hdu.data = std
    hdu.header = header
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tempfilen))
    hdulist.close()

## Optional (needed in my case since I use DCR to remove cosmic rays):
### Replace all the infinities in the standard deviation image by 150 so that we can still remove the cosmic rays
 

In [ ]:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tempfile = os.path.splitext(tempfile)[0]
    filen = tempfile+ '_std_shifted.fits'
    tempfilen = tempfile + '_std_infrep_shifted.fits'
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',filen)
    hdulist = fits.open(filename)
    header = hdulist[0].header
    std = hdulist[0].data
    std[np.isinf(std)]=150
    hdulist.close()
    
    hdu = fits.PrimaryHDU()
    hdu.data = std
    hdu.header = header
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tempfilen))
    hdulist.close()

## Now run data through DCR --> called _shifted_bgsub_cleaned.fits 

## Run error fits through DCR --> called _std_shifted_cleaned.fits

### Once DCR has run properly, replace all the 150 values (that had replaced np.inf) with np.inf again so that everything will plot well in DS9

In [ ]:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tempfile = os.path.splitext(tempfile)[0]
    filen = tempfile+ '_std_infrep_shifted_cleaned.fits'
    tempfilen = tempfile + '_std_shifted_cleaned.fits'
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',filen)
    hdulist = fits.open(filename)
    header = hdulist[0].header
    std = hdulist[0].data
    std[std==150]=np.inf
    hdulist.close()
    
    hdu = fits.PrimaryHDU()
    hdu.data = std
    hdu.header = header
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tempfilen))
    hdulist.close()

## Produce normalized coadded image and normalized coadded standard deviation image

In [ ]:
#produce renormalized coadded image
#coadd the images that have had cosmic rays removed:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

tempname = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted','sci-b150909_1089_shifted_bgsub_cleaned.fits') 
hdulist2 = fits.open(tempname)
scidata0 = hdulist2[0].data
temparray = np.zeros_like(scidata0)

time = 0
for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tfile = os.path.splitext(tempfile)[0]
    tfilen = tfile + '_shifted_bgsub_cleaned.fits'
    
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tfilen)
    hdulist = fits.open(filename)
    scidata = hdulist[0].data
    header = hdulist[0].header
    time += int(header['TTIME'])
    temparray += scidata
    hdulist.close()

hdu = fits.PrimaryHDU()
hdu.data = temparray/time
hdu.header = hdulist2[0].header
hdulist = fits.HDUList([hdu])
#hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','camille_x','Desktop','coadded_LRIS_Slug_noCR_normalized.fits'))
hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted','coadded_LRIS_Slug_noCR_normalized.fits'))
hdulist.close()
hdulist2.close()  

In [ ]:
#produce renormalized coadded standard deviation array
#coadd the images that have had cosmic rays removed:
filenames = np.array(['sci-b150909_1089.fits','sci-b150909_1090.fits','sci-b150909_1091.fits','sci-b150909_1092.fits','sci-b150909_1093.fits','sci-b150909_1094.fits','sci-b150909_1095.fits','sci-b150909_1096.fits','sci-b150909_1097.fits'])

tempname = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted','sci-b150909_1089_shifted_bgsub_cleaned.fits') 
hdulist2 = fits.open(tempname)
scidata0 = hdulist2[0].data
temparray = np.zeros_like(scidata0)

time = 0
for fn in xrange(len(filenames)):
    tempfile = filenames[fn]
    tfile = os.path.splitext(tempfile)[0]
    tfilen = tfile + '_std_shifted_cleaned.fits'
    
    filename = os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted',tfilen)
    hdulist = fits.open(filename)
    stddata = hdulist[0].data
    header = hdulist[0].header
    time += int(header['TTIME'])
    temparray += np.power(stddata,2)
    hdulist.close()

hdu = fits.PrimaryHDU()
hdu.data = np.sqrt(temparray)/time
hdu.header = hdulist2[0].header
hdulist = fits.HDUList([hdu])
#hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','camille_x','Desktop','coadded_LRIS_Slug_noCR_normalized.fits'))
hdulist.writeto(os.path.join(os.path.expanduser('~'),'Dropbox','LowRedux','2015sept','mask_slug_n2_v2','Blue','Science_Slug_shifted','coadded_LRIS_Slug_noCR_normalized_std.fits'))
hdulist.close()
hdulist2.close()  

#note that the coadded image is in e-/s